In [3]:
# Import necessary libraries
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


# Import CRUD Python module
from Module4 import AnimalShelter

# Connect to MongoDB database
username = "accuser"
password = "password"
shelter = AnimalShelter(username, password)

# Read data from database and store in Pandas DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Initialize Dash app
app = JupyterDash('SimpleExample')

# Load image and add it to app layout
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
app.layout = html.Div([
    # Add logo image to layout
    html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())), href='www.snhu.edu'),
    # Add heading and horizontal line
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # Add radio buttons for filtering data by rescue type
    html.Div([
        dcc.RadioItems(
            id='radio',
            options=[{'label': i, 'value': i} for i in ['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset']],
            value='Water Rescue'
        )
    ]),
    html.Hr(),
    # Add data table to layout
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    # Add chart and map to layout
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
                        id='map-id',
            className='col s12 m6',
        )
    ]),
    # Add credit line
    html.P("Caleb VanDerMaas 2022")
])

# Define callback function to filter data and update chart and map when radio button selection is changed
@app.callback([Output('datatable-id','data'),
               Output('graph-id','children'),
               Output('map-id','children')],
              [Input('radio','value')])
def update_table_graph_map(value):
    if value == 'Reset':
        df_new = df
    else:
        df_new = df[df['Rescue Type'] == value]
    graph = dcc.Graph(id='pie', figure=px.pie(df_new, values='Count', names='Breed'))
    center = [df_new['Latitude'].mean(), df_new['Longitude'].mean()]
    map_ = dl.Map(id='my-map', center=center, zoom=4)
    return df_new.to_dict('records'), graph, map_

# Define callback function to update chart and map when a row in the data table is selected
@app.callback([Output('graph-id','children'),
               Output('map-id','children')],
              [Input('datatable-id','selected_rows'),
               Input('hidden-div','children')])
def update_graph_map(selected_rows, hidden):
    if selected_rows is None:
        selected_rows = [0]
    df_new = df.iloc[selected_rows]
    graph = dcc.Graph(id='pie', figure=px.pie(df_new, values='Count', names='Breed'))
    center = [df_new['Latitude'].mean(), df_new['Longitude'].mean()]
    map_ = dl.Map(id='my-map', center=center, zoom=4)
    return graph, map_

# Run app
app.run_server(mode='inline')



TypeError: __init__() takes 1 positional argument but 3 were given